# Fed-BioMed secure aggregation tutorial

## Example experimentation setup

This part contains setup of a basic example for Fed-BioMed. At this point, nothing is specific to secure aggregation.

### Start the network
Before running this notebook, start the network with `./scripts/fedbiomed_run network`

### Setting nodes up
It is necessary to previously configure ** at least two nodes**:
1. `./scripts/fedbiomed_run node config config_node1.ini add` (respectively for the second node: `./scripts/fedbiomed_run node config config_node2.ini add`)
  * Select option 2 (default) to add MNIST to the node
  * Confirm default tags by hitting "y" and ENTER
  * Pick the folder where MNIST is downloaded (this is due to a pytorch issue https://github.com/pytorch/vision/issues/3549)
  * Data must have been added (if you get a warning saying that data must be unique is because it's been already added)
  
2. Check that your data has been added by executing `./scripts/fedbiomed_run config config_node1.ini node list`
3. Run the node using `./scripts/fedbiomed_run config_node1.ini node run`. Wait until you get `Starting task manager`. it means you are online.

### Define an experiment model and parameters"

Declare a torch training plan MyTrainingPlan class to send for training on the node

In [1]:
import torch
import torch.nn as nn
from fedbiomed.common.training_plans import TorchTrainingPlan
from fedbiomed.common.data import DataManager
from torchvision import datasets, transforms


# Here we define the model to be used. 
# You can use any class name (here 'Net')
class MyTrainingPlan(TorchTrainingPlan):
    
    # Defines and return model 
    def init_model(self, model_args):
        return self.Net(model_args = model_args)
    
    # Defines and return optimizer
    def init_optimizer(self, optimizer_args):
        return torch.optim.Adam(self.model().parameters(), lr = optimizer_args["lr"])
    
    # Declares and return dependencies
    def init_dependencies(self):
        deps = ["from torchvision import datasets, transforms"]
        return deps
    
    class Net(nn.Module):
        def __init__(self, model_args):
            super().__init__()
            self.conv1 = nn.Conv2d(1, 32, 3, 1)
            self.conv2 = nn.Conv2d(32, 64, 3, 1)
            self.dropout1 = nn.Dropout(0.25)
            self.dropout2 = nn.Dropout(0.5)
            self.fc1 = nn.Linear(9216, 128)
            self.fc2 = nn.Linear(128, 10)

        def forward(self, x):
            x = self.conv1(x)
            x = F.relu(x)
            x = self.conv2(x)
            x = F.relu(x)
            x = F.max_pool2d(x, 2)
            x = self.dropout1(x)
            x = torch.flatten(x, 1)
            x = self.fc1(x)
            x = F.relu(x)
            x = self.dropout2(x)
            x = self.fc2(x)


            output = F.log_softmax(x, dim=1)
            return output

    def training_data(self, batch_size = 48):
        # Custom torch Dataloader for MNIST data
        transform = transforms.Compose([transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))])
        dataset1 = datasets.MNIST(self.dataset_path, train=True, download=False, transform=transform)
        train_kwargs = {'batch_size': batch_size, 'shuffle': True}
        return DataManager(dataset=dataset1, **train_kwargs)
    
    def training_step(self, data, target):
        output = self.model().forward(data)
        loss   = torch.nn.functional.nll_loss(output, target)
        return loss


This group of arguments correspond respectively:
* `model_args`: a dictionary with the arguments related to the model (e.g. number of layers, features, etc.). This will be passed to the model class on the node side.
* `training_args`: a dictionary containing the arguments for the training routine (e.g. batch size, learning rate, epochs, etc.). This will be passed to the routine on the node side.

**NOTE:** typos and/or lack of positional (required) arguments will raise error. 🤓

In [2]:
model_args = {}

training_args = {
    'batch_size': 48, 
    'optimizer_args': {
        "lr" : 1e-3
    },
    'epochs': 1, 
    'dry_run': False,  
    'batch_maxnum': 100 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}

### Declare and run the experiment

In [7]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage
from fedbiomed.researcher.secagg import SecureAggregation
tags =  ['#MNIST', '#dataset']
rounds = 2

exp = Experiment(tags=tags,
                 model_args=model_args,
                 training_plan_class=MyTrainingPlan,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None,
                 secagg=True, # or custom SecureAggregation(active=<bool>, clipping_range=<int>, timeout=<int>)
                 save_breakpoints=True)

2023-06-19 15:26:59,848 fedbiomed INFO - Searching dataset with data tags: ['#MNIST', '#dataset'] for all nodes
2023-06-19 15:27:09,864 fedbiomed INFO - Node selected for training -> node_cc8823f5-b234-447e-a41a-c2ad64a1072f
2023-06-19 15:27:09,865 fedbiomed INFO - Node selected for training -> node_8d470259-46b9-46cd-adba-3d050899da73
2023-06-19 15:27:09,868 fedbiomed INFO - Checking data quality of federated datasets...
Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
2023-06-19 15:27:09,887 fedbiomed DEBUG - using native torch optimizer
2023-06-19 15:27:09,889 fedbiomed DEBUG - Model file has been saved: /workspaces/Projects/fedbiomed/var/experiments/Experiment_0009/my_model_0b8861d1-b1fc-41bb-b8f7-19b04df97585.py
2023-06-19 15:27:09,907 fedbiomed DEBUG - HTTP POST request of file /workspaces/Projects/fedbiomed/var

### Access secure aggregation context

Please use the attribute `secagg` to verify secure aggregation is set as active

In [8]:
print("Is using secagg: ", exp.secagg.active)

Is using secagg:  True


It is also possible to check secure aggregation context using `secagg` attribute. Since secure aggregation context negotiation will occur during experiment run, context and id should be `None` at this point.

In [9]:
print("Secagg Biprime ", exp.secagg.biprime)
print("Secagg Servkey ", exp.secagg.servkey)

Secagg Biprime  None
Secagg Servkey  None


Run the experiment, using secure aggregation. Secure aggregation context will be created before the first training round and it is going to be updated before each round when new nodes are added or removed to the experiment. 

In [ ]:
exp.run(increase=True)

2023-06-19 15:27:21,056 fedbiomed DEBUG - researcher_73f4aba0-45be-4b82-ae3f-ce4cfd2dea68
2023-06-19 15:27:21,058 fedbiomed DEBUG - researcher_73f4aba0-45be-4b82-ae3f-ce4cfd2dea68
2023-06-19 15:27:21,059 fedbiomed DEBUG - Secagg context for default_biprime0 is already existing on researcher researcher_id='researcher_73f4aba0-45be-4b82-ae3f-ce4cfd2dea68'
2023-06-19 15:27:21,063 fedbiomed INFO - INFO
					 NODE node_cc8823f5-b234-447e-a41a-c2ad64a1072f
					 MESSAGE: Node secagg context element for default_biprime0 is already existing for job None
-----------------------------------------------------------------
2023-06-19 15:27:21,104 fedbiomed INFO - INFO
					 NODE node_8d470259-46b9-46cd-adba-3d050899da73
					 MESSAGE: Node secagg context element for default_biprime0 is already existing for job None
-----------------------------------------------------------------
2023-06-19 15:27:22,062 fedbiomed DEBUG - researcher_73f4aba0-45be-4b82-ae3f-ce4cfd2dea68
2023-06-19 15:27:22,065 fedbio

2023-06-19 15:27:26,476 fedbiomed INFO - TRAINING 
					 NODE_ID: node_8d470259-46b9-46cd-adba-3d050899da73 
					 Round 1 Epoch: 1 | Iteration: 10/100 (10%) | Samples: 480/4800
 					 Loss: 1.417369 
					 ---------
2023-06-19 15:27:27,087 fedbiomed INFO - TRAINING 
					 NODE_ID: node_cc8823f5-b234-447e-a41a-c2ad64a1072f 
					 Round 1 Epoch: 1 | Iteration: 10/100 (10%) | Samples: 480/4800
 					 Loss: 1.322947 
					 ---------
2023-06-19 15:27:27,377 fedbiomed INFO - TRAINING 
					 NODE_ID: node_8d470259-46b9-46cd-adba-3d050899da73 
					 Round 1 Epoch: 1 | Iteration: 20/100 (20%) | Samples: 960/4800
 					 Loss: 0.923158 
					 ---------
2023-06-19 15:27:28,005 fedbiomed INFO - TRAINING 
					 NODE_ID: node_cc8823f5-b234-447e-a41a-c2ad64a1072f 
					 Round 1 Epoch: 1 | Iteration: 20/100 (20%) | Samples: 960/4800
 					 Loss: 0.708664 
					 ---------
2023-06-19 15:27:28,347 fedbiomed INFO - TRAINING 
					 NODE_ID: node_8d470259-46b9-46cd-adba-3d050899da73 
					 Round 1 Epoch: 1 

Display context after running one round of training. 

In [ ]:
print("Secagg Biprime context: ", exp.secagg.biprime.context)
print("Secagg Servkey context: ", exp.secagg.servkey.context)

#### Changes in experiment triggers re-creation of secure aggregation context

The changes that re-create jobs like adding new node to the experiment will trigger automatic secure aggregation re-setup for the next round.  

In [ ]:
# sends new dataset search request
from fedbiomed.researcher.strategies import DefaultStrategy
from fedbiomed.researcher.aggregators.fedavg import FedAverage
exp.set_training_data(None, True)
exp.set_strategy(DefaultStrategy)
exp.set_aggregator(FedAverage)
exp.set_job()

In [ ]:
exp.run_once(increase=True)

### Changing arguments of secure aggregation

Setting `secagg` argument `True` in `Experiment` creates a default `SecureAggregation` instance. Additionally, It is also possible to create `SecureAggregation` instance and pass it as an argument. Here are the arguments that can be set for the `SecureAggregation`

- `active`: `True` if the round will use secure aggregation. Default is `True`
- `clipping_range`: Clipping range that is goingto be use for quantization of model parameters. Default clipping range is `3`. However, some models can have model weigths greater than `3`. If clipping range is exceeded during the encryption on the nodes, `Experiment` will log a warning message. In such cases, you can provide a higher clipping range through the argument `clipping_range`.
- `timeout`: Timeout is the maximum amount of time, in seconds, that the experiment will wait for responses from all parties during secure aggregation setup. Since secure aggregation context depends on network communication and multi-party computation, this argument allows to set higher timeout for larger context setups, or vice versa. 

In [ ]:
from fedbiomed.researcher.secagg import SecureAggregation
secagg = SecureAggregation(
    active=True, 
    clipping_range=100,
    timeout=15
    
)
exp.set_secagg(secagg=secagg)


In [ ]:
exp.run_once(increase=True)

### Load experiment from a breakpoint

Once a breakpoint is loadded if the context is already exsiting there won't be context setup. 

In [ ]:
loaded_exp = Experiment.load_breakpoint()
loaded_exp.info()

In [ ]:
loaded_exp.run_once(increase=True)

2023-06-19 15:26:33,226 fedbiomed INFO - CRITICAL
					 NODE node_8d470259-46b9-46cd-adba-3d050899da73
					 MESSAGE: Node stopped in signal_handler, probably by user decision (Ctrl C)
-----------------------------------------------------------------
2023-06-19 15:26:35,064 fedbiomed INFO - CRITICAL
					 NODE node_cc8823f5-b234-447e-a41a-c2ad64a1072f
					 MESSAGE: Node stopped in signal_handler, probably by user decision (Ctrl C)
-----------------------------------------------------------------
2023-06-19 15:26:45,543 fedbiomed INFO - INFO
					 NODE node_8d470259-46b9-46cd-adba-3d050899da73
					 MESSAGE: Starting task manager
-----------------------------------------------------------------
2023-06-19 15:26:53,902 fedbiomed INFO - INFO
					 NODE node_cc8823f5-b234-447e-a41a-c2ad64a1072f
					 MESSAGE: Starting task manager
-----------------------------------------------------------------
